In [ ]:
!pwd 


In [ ]:
import firedrake
import icepack

In [ ]:
import geojson

outline_filename = "mergedfile.geojson"
with open(outline_filename, "r") as outline_file:
    outline = geojson.load(outline_file)

In [ ]:
coords = list(geojson.utils.coords(outline))

In [ ]:
import pyproj
lat_lon = pyproj.CRS(4326)
utm8 = pyproj.CRS(32608)
transformer = pyproj.Transformer.from_crs(lat_lon, utm8)
print(transformer.transform(coords[0][0], coords[0][1]))
print(transformer.transform(coords[0][1], coords[0][0]))

In [ ]:
print(outline["crs"]["properties"]["name"])

In [ ]:
import numpy as np

δ = 50e3
coords = np.array(list(geojson.utils.coords(outline)))
xmin, xmax = coords[:, 0].min() - δ, coords[:, 0].max() + δ
ymin, ymax = coords[:, 1].min() - δ, coords[:, 1].max() + δ

In [ ]:
import icepack.plot

fig, axes = icepack.plot.subplots()

for feature in outline["features"]:
    for line_string in feature["geometry"]["coordinates"]:
        xs = np.array(line_string)
        axes.plot(xs[:, 0], xs[:, 1], linewidth=2)

axes.set_xlabel("longitude");

Initial mesh generation.

In [ ]:
from meshpy import triangle

In [ ]:
geometry = icepack.meshing.collection_to_triangle(outline)

In [ ]:
triangle_mesh = triangle.build(geometry, max_volume=4e-5)

In [ ]:
mesh = icepack.meshing.triangle_to_firedrake(triangle_mesh)

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 2)
V = firedrake.VectorFunctionSpace(mesh, "CG", 2)

In [ ]:
fig, axes = icepack.plot.subplots()
firedrake.triplot(mesh, axes=axes)
#axes.legend()

In [ ]:
S = firedrake.FunctionSpace(mesh, "DG", 0)
areas = firedrake.project(firedrake.CellVolume(mesh), S)

In [ ]:
fig, axes = icepack.plot.subplots()
colors = firedrake.tripcolor(areas, axes=axes)
fig.colorbar(colors);

Mesh refinement.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
expr = firedrake.conditional(
    x[1] <= 62.9,
    areas / 2,
    areas
)
desired_areas = firedrake.project(expr, S)

In [ ]:
fig, axes = icepack.plot.subplots()
colors = firedrake.tripcolor(desired_areas, axes=axes)
fig.colorbar(colors);

In [ ]:
triangle_mesh.element_volumes.setup()
for index, area in enumerate(desired_areas.dat.data_ro):
    triangle_mesh.element_volumes[index] = area

In [ ]:
refined_triangle_mesh = triangle.refine(triangle_mesh)

In [ ]:
fine_mesh = icepack.meshing.triangle_to_firedrake(refined_triangle_mesh)

In [ ]:
fig, axes = icepack.plot.subplots()
firedrake.triplot(fine_mesh, axes=axes);